In [0]:
import json

from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

####Join Static data with Streaming Data

In [0]:
# create a schema for static data
static_schema = StructType([StructField("name", StringType(), True),
                            StructField("year", IntegerType(), True),
                            StructField("director", StringType(), True),
                            StructField("writer", StringType(), True),
                            StructField("star", StringType(), True)
                           ])

In [0]:
# read static data from local DBFS into spark dataframe. This will be our Static data source for joining
static_data = spark.read\
                   .format("csv")\
                   .option("header", "true")\
                   .schema(static_schema )\
                   .load("dbfs:/FileStore/datasets/movies/")

static_data.display()

name,year,director,writer,star
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro
Superman II,1980,Richard Lester,Jerry Siegel,Gene Hackman


In [0]:
# connect to Event Hub as a source for streaming data

primaryKey = "Endpoint=sb://db-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=86ATi9XElcqYcspgAo8KNdXzMiWsSJbj++AEhA35NY8="
entityPath = "EntityPath=eventhub-ratings"

connectionString = primaryKey + ";" + entityPath

ehConf = {}

#this configurations will read data from the beginning of the source

startOffset = "-1"

startingEventPosition = {
  "offset": startOffset,  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
ehConf["eventhubs.startingPosition"] = json.dumps(startingEventPosition)

In [0]:
# create schema for reading data
streaming_schema = StructType([StructField("name", StringType(), True),
                               StructField("rating", StringType(), True),
                               StructField("score", FloatType(), True)])

In [0]:
# Before running this step run the source notebook ("RatingsSource") that will push data to the Event hub
# read stream data from Event Hub (comming in json format)
streaming_data = spark.readStream \
                      .format("eventhubs") \
                      .options(**ehConf) \
                      .load()

In [0]:
streaming_data = streaming_data.selectExpr("CAST(body AS STRING)")

streaming_data.display()

body
"{ ""name"": ""Star Wars: Episode V - The Empire Strikes Back"", ""rating"": ""PG"", ""score"": 8.2}"
"{ ""name"": ""Fame"", ""rating"": ""R"", ""score"": 6.6}"
"{ ""name"": ""Friday the 13th"", ""rating"": ""R"", ""score"": 6.1}"
"{ ""name"": ""Lagaan"", ""rating"": ""PG"", ""score"": 9.2 }"
"{ ""name"": ""Andhadhun"", ""rating"": ""PG"", ""score"": 8.4 }"
"{ ""name"": ""Airplane!"",""rating"": ""PG"", ""score"": 7.7 }"
"{ ""name"": ""Caddyshack"", ""rating"": ""R"", ""score"": 7.3 }"
"{ ""name"": ""Friday the 13th"", ""rating"": ""R"", ""score"": 6.4}"
"{ ""name"": ""The Blues Brothers"", ""rating"": ""R"", ""score"": 7.9}"
"{ ""name"": ""Raging Bull"", ""rating"": ""R"", ""score"": 8.2 }"


In [0]:
# extract column data from json, using defined schema
streaming_data = streaming_data.select(from_json(col("body").cast("string"), streaming_schema)) \
                          .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                          .select(col('data.*'))

streaming_data.display()

name,rating,score
Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Fame,R,6.6
Friday the 13th,R,6.1
Lagaan,PG,9.2
Andhadhun,PG,8.4
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2


In [0]:
# Outer Join - all elements of the batch with all elements of the stream (without specifying column to join on)
outer_join = static_data.join(streaming_data)

outer_join.display()

name,year,director,writer,star,name,rating,score
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Superman II,1980,Richard Lester,Jerry Siegel,Gene Hackman,Star Wars: Episode V - The Empire Strikes Back,PG,8.2
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,Fame,R,6.6


In [0]:
# right outer join (streaming data on the right)
right_outer_join = static_data.join(streaming_data, on=["name"], how="right_outer")

right_outer_join.display()

name,year,director,writer,star,rating,score
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Fame,null,null,null,null,R,6.6
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.1
Lagaan,null,null,null,null,PG,9.2
Andhadhun,null,null,null,null,PG,8.4
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,PG,7.7
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,R,7.3
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.4
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi,R,7.9
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro,R,8.2


In [0]:
# left outer join isn't possible without watermark when stream data are on the right (spark  must have explicit set of records) 
left_outer_join_rightstream = static_data.join(streaming_data, on=["name"], how="left_outer")

left_outer_join_rightstream.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1434252330829294> in <module> 
 2 left_outer_join_rightstream = static_data . join ( streaming_data , on = [ "name" ] , how = "left_outer" ) 
 3 
 ----> 4 left_outer_join_rightstream . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 34 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 35 """
 ---> 36 display ( df , * args , ** kwargs ) 
 37 
 38 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 986 # exists and then if it does, whether it is actually streaming. 
 987 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 --> 988 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 989 else : 
 990 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 117 .DisplayHelper.getStreamName())
 118 
 --> 119 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 120 kwargs.get('checkpointLocation'))
 121 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : LeftOuter join with a streaming DataFrame/Dataset on the right and a static DataFrame/Dataset on the left is not supported;
Join LeftOuter, (name#114 = name#328)
:- Relation [name#114,year#115,director#116,writer#117,star#118] csv
+- Project [data#326.name AS name#328, data#326.rating AS rating#329, data#326.score AS score#330]
 +- Project [from_json(CAST(body AS STRING))#324 AS data#326]
 +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#271 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#324]
 +- Project [cast(body#253 as string) AS body#271]
 +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@742efd71,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> +uQzV6R/bdPVogbaEgbQDO+B0aG+fd410pV+hHSUA7gFWco6+UvWjraCVX6ygZXvuQimx5knuCAiwcaN7A+z6arVqQVvtoRhTILa5RYBZOiIaCZkqwOr/8jLDp1r2QEP0+UC4FSAzw9RdnaPKq20IZ3StwwX3mONr6RJVQVfvzpNN+SGVNQ5GAgwjf448kk2BpHF6N2c5+ICSraC4NPEO73YRC0IFghXQWvbAH5eR7t6ML2MSo5K/raO/RejNSVz1yqQ6CUD7o90OLplIGrQuA==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#253, partition#254, offset#255, sequenceNumber#256L, enqueuedTime#257, publisher#258, partitionKey#259, properties#260, systemProperties#261]

In [0]:
# basicly, the same as above, just mirrorred. Returns error also
right_outer_join_leftstream = streaming_data.join(static_data, on=["name"], how="right_outer")

right_outer_join_leftstream.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1434252330829295> in <module> 
 2 right_outer_join_leftstream = streaming_data . join ( static_data , on = [ "name" ] , how = "right_outer" ) 
 3 
 ----> 4 right_outer_join_leftstream . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 34 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 35 """
 ---> 36 display ( df , * args , ** kwargs ) 
 37 
 38 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 986 # exists and then if it does, whether it is actually streaming. 
 987 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 --> 988 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 989 else : 
 990 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 117 .DisplayHelper.getStreamName())
 118 
 --> 119 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 120 kwargs.get('checkpointLocation'))
 121 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : RightOuter join with a streaming DataFrame/Dataset on the left and a static DataFrame/DataSet on the right not supported;
Join RightOuter, (name#328 = name#114)
:- Project [data#326.name AS name#328, data#326.rating AS rating#329, data#326.score AS score#330]
: +- Project [from_json(CAST(body AS STRING))#324 AS data#326]
: +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#271 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#324]
: +- Project [cast(body#253 as string) AS body#271]
: +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@742efd71,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> +uQzV6R/bdPVogbaEgbQDO+B0aG+fd410pV+hHSUA7gFWco6+UvWjraCVX6ygZXvuQimx5knuCAiwcaN7A+z6arVqQVvtoRhTILa5RYBZOiIaCZkqwOr/8jLDp1r2QEP0+UC4FSAzw9RdnaPKq20IZ3StwwX3mONr6RJVQVfvzpNN+SGVNQ5GAgwjf448kk2BpHF6N2c5+ICSraC4NPEO73YRC0IFghXQWvbAH5eR7t6ML2MSo5K/raO/RejNSVz1yqQ6CUD7o90OLplIGrQuA==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#253, partition#254, offset#255, sequenceNumber#256L, enqueuedTime#257, publisher#258, partitionKey#259, properties#260, systemProperties#261]
+- Relation [name#114,year#115,director#116,writer#117,star#118] csv

In [0]:
# left outer join with streaming data on the left
left_outer_join = streaming_data.join(static_data, on=["name"], how="left_outer")

left_outer_join.display()

name,rating,score,year,director,writer,star
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Fame,R,6.6,null,null,null,null
Friday the 13th,R,6.1,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer
Lagaan,PG,9.2,null,null,null,null
Andhadhun,PG,8.4,null,null,null,null
Airplane!,PG,7.7,1980,Jim Abrahams,Jim Abrahams,Robert Hays
Caddyshack,R,7.3,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase
Friday the 13th,R,6.4,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer
The Blues Brothers,R,7.9,1980,John Landis,Dan Aykroyd,John Belushi
Raging Bull,R,8.2,1980,Martin Scorsese,Jake LaMotta,Robert De Niro


In [0]:
# inner join
inner_join = static_data.join(streaming_data, on=["name"], how="inner")

inner_join.display()

name,year,director,writer,star,rating,score
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.1
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,PG,7.7
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,R,7.3
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.4
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi,R,7.9
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro,R,8.2
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.1
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2


In [0]:
# combine joins with other operations
selected_join = static_data.join(streaming_data, on=["name"], how="inner")\
                           .select("name", "director", "star", "score") 

selected_join.display()

name,director,star,score
Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,8.2
Friday the 13th,Sean S. Cunningham,Betsy Palmer,6.1
Airplane!,Jim Abrahams,Robert Hays,7.7
Caddyshack,Harold Ramis,Chevy Chase,7.3
Friday the 13th,Sean S. Cunningham,Betsy Palmer,6.4
The Blues Brothers,John Landis,John Belushi,7.9
Raging Bull,Martin Scorsese,Robert De Niro,8.2
Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,8.2
Friday the 13th,Sean S. Cunningham,Betsy Palmer,6.1
Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,8.2


In [0]:
top_scorers = selected_join.select("name", "score").where(selected_join.score > 8)

top_scorers.display()

name,score
Star Wars: Episode V - The Empire Strikes Back,8.2
Raging Bull,8.2
Star Wars: Episode V - The Empire Strikes Back,8.2
Star Wars: Episode V - The Empire Strikes Back,8.2


In [0]:
# try to perform inner join on column that present only in the streaming source. It will result into en error
inner_join_error = static_data.join(streaming_data, on=["star"], how="inner")

inner_join_error.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1434252330829302> in <module> 
 1 # try to perform inner join on column that present only in the streaming source 
 ----> 2 inner_join_error = static_data . join ( streaming_data , on = [ "star" ] , how = "inner" ) 
 3 
 4 inner_join_error . display ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in join (self, other, on, how) 
 1378 on = self . _jseq ( [ ] ) 
 1379 assert isinstance ( how , str ) , "how should be a string" 
 -> 1380 jdf = self . _jdf . join ( other . _jdf , on , how ) 
 1381 return DataFrame ( jdf , self . sql_ctx ) 
 1382 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : USING column `star` cannot be resolved on the right side of the join. The right-side columns: [name, rating, score]